as shown in Figure 10.4, in which each bucket may manage a collection of items that are sent to a specific index by the hash function. (To save space, an empty bucket may be replaced by None.)

<img src='img/桶数组.jpg' style="zoom:20%">

## Hash Functions

The goal of a hash function, h, is to map each key k to an integer in the range $[0,N −1]$, where $N$ is the capacity of the bucket array for a hash table. Equipped with such a hash function, $h$, the main idea of this approach is to use the hash function value, $h(k)$, as an index into our bucket array, $A$, instead of the key $k$ (which may not be appropriate for direct use as an index). That is, we store the item $(k,v)$ in the bucket $A[h(k)]$.

It is common to view the evaluation of a hash function, $h(k)$, as consisting of two portions—a hash code that maps a key k to an integer, and a compression function that maps the hash code to an integer within a range of indices, $[0,N −1]$, for a bucket array.(See Figure 10.5)

<img src='img/哈希函数的两个部分.jpg' style="zoom:20%">

### 哈希码(hash code)

The first action that a hash function performs is to take an arbitrary key k in our map and compute an integer that is called the hash code for k; this integer need not be in the range $[0,N −1]$, and may even be negative. We desire that the set of hash codes assigned to our keys should avoid collisions as much as possible. For if the hash codes of our keys cause collisions, then there is no hope for our compression function to avoid them.

### 压缩函数(compression function)

The hash code for a key k will typically not be suitable for immediate use with a bucket array, because the integer hash code may be negative or may exceed the capacity of the bucket array. Thus, once we have determined an integer hash code for a key object k, there is still the issue of mapping that integer into the range [0,N−1]. This computation, known as a compression function, is the second action performed as part of an overall hash function. A good compression function is one that minimizes the number of collisions for a given set of distinct hash codes.

#### 划分方法

A simple compression function is the division method, which maps an integer i to

$$ i \, \mathrm{mod}  \, N $$

where $N$, the size of the bucket array, is a fixed positive integer. Additionally, if we take $ N $ to be a prime number, then this compression function helps “spread out” the distribution of hashed values. Indeed, if $ N $ is not prime, then there is greater risk that patterns in the distribution of hash codes will be repeated in the distribution of hash values, thereby causing collisions. For example, if we insert keys with hash codes {200,205,210,215,220, . . . ,600} into a bucket array of size 100, then each hash code will collide with three others. But if we use a bucket array of size 101, then there will be no collisions. If a hash function is chosen well, it should ensure that the probability of two different keys getting hashed to the same bucket is $1/N$. Choosing $N$ to be a prime number is not always enough, however, for if there is a repeated pattern of hash codes of the form $pN +q$ for several different $p$’s, then there will still be collisions.

#### MAD方法
A more sophisticated compression function, which helps eliminate repeated patterns in a set of integer keys, is the Multiply-Add-and-Divide (or “MAD”) method. This method maps an integer i to

$$ [(ai + b) \, \mathrm{mod} \, p ] \, \mathrm{mod} \, N $$


where $ N $ is the size of the bucket array, $ p $ is a prime number larger than $ N $, and $a$ and $b$ are integers chosen at random from the interval $[0, p−1]$, with $a > 0$. This compression function is chosen in order to eliminate repeated patterns in the set of hash codes and get us closer to having a “good” hash function, that is, one such that the probability any two different keys collide is $ 1/N $. This good behavior would be the same as we would have if these keys were “thrown” into $A$ uniformly at random.


### 冲突解决方案

The main idea of a hash table is to take a bucket array, $A$, and a hash function, $h$, and use them to implement a map by storing each item $(k,v)$ in the “bucket” $A[h(k)]$. This simple idea is challenged, however, when we have two distinct keys, $k1$ and $k2$, such that $h(k1) = h(k2)$. The existence of such collisions prevents us from simply inserting a new item $(k,v)$ directly into the bucket $A[h(k)]$. It also complicates our procedure for performing insertion, search, and deletion operations.

#### 分离链表(separate chaining)

A simple and efficient way for dealing with collisions is to have each bucket $A[j]$ store its own secondary container, holding items $(k,v)$ such that $h(k) = j$. This collision resolution rule is known as separate chaining, and is illustrated in Figure 10.6.

<img src='img/分离链表.jpg' style="zoom:20%">

In the worst case, operations on an individual bucket take time proportional to the size of the bucket. Assuming we use a good hash function to index the n items of our map in a bucket array of capacity $N$, the expected size of a bucket is $n/N$. Therefore, if given a good hash function, the core map operations run in $O(n/N)$. The ratio $ \lambda = n/N$, called the load factor of the hash table, should be bounded by a small constant, preferably below 1. As long as $\lambda$ is $O(1)$, the core operations on the hash table run in $O(1)$ expected time.


#### 开放寻址(open addressing)

The separate chaining rule has many nice properties, such as affording simple implementations of map operations, but it nevertheless has one slight disadvantage: It requires the use of an auxiliary data structure—a list—to hold items with colliding keys. If space is at a premium (for example, if we are writing a program for a small handheld device), then we can use the alternative approach of always storing each item directly in a table slot. This approach saves space because no auxiliary structures are employed, but it requires a bit more complexity to deal with collisions. There are several variants of this approach, collectively referred to as open addressing schemes(线性探测,平方探测,二次探测等), which we discuss next. Open addressing requires that the load factor is always at most 1 and that items are stored directly in the cells of the bucket array itself.

A simple method for collision handling with open addressing is linear probing. With this approach, if we try to insert an item $(k,v)$ into a bucket $A[j]$ that is already occupied, where $j = h(k)$, then we next try $A[(j+1) \, \mathrm{mod} \, N]$. If $A[(j+1) \, \mathrm{mod} \, N]$ is also occupied, then we try $A[(j+2) \, \mathrm{mod} \, N]$, and so on, until we find an empty bucket that can accept the new item. Once this bucket is located, we simply insert the item there. Of course, this collision resolution strategy requires that we change the implementation when searching for an existing key—the first step of all \_\_getitem\_\_, \_\_setitem\_\_, or \_\_delitem\_\_ operations. In particular, to attempt to locate an item with key equal to k, we must examine consecutive slots, starting from $A[h(k)]$, until we either find an item with that key or we find an empty bucket. (See Figure 10.7.) The name “linear probing” comes from the fact that accessing a cell of the bucket array can be viewed as a “probe.”

<img src='img/线性探测.jpg' style="zoom:20%">


To implement a deletion, we cannot simply remove a found item from its slot in the array. For example, after the insertion of key 15 portrayed in Figure 10.7, if the item with key 37 were trivially deleted, a subsequent search for 15 would fail because that search would start by probing at index 4, then index 5, and then index 6, at which an empty cell is found. A typical way to get around this difficulty is to replace a deleted item with a special “available” marker object. With this special marker possibly occupying spaces in our hash table, we modify our search algorithm so that the search for a key k will skip over cells containing the available marker and continue probing until reaching the desired item or an empty bucket (or returning back to where we started from). Additionally, our algorithm for \_\_setitem\_\_ should remember an available cell encountered during the search for $k$, since this is a valid place to put a new item $(k,v)$, if no existing item is found.

Although use of an open addressing scheme can save space, linear probing suffers from an additional disadvantage. It tends to cluster the items of a map into contiguous runs, which may even overlap (particularly if more than half of the cells in the hash table are occupied). Such contiguous runs of occupied hash cells cause searches to slow down considerably.